In [ ]:
import utente
import pandas as pd
import numpy as np
import pandas_profiling as pf
pd.options.display.max_columns = 100
pd.options.display.max_rows = 250

In [ ]:
# Carregamento estrutura dados (passar parâmetro forceFetch = True para forçar leitura BD >> Demorado!)
listaUtentes = utente.ListaUtentes(forceFetch = False, sqlLimit = 99999999999) # Carrega dados locais a partir de PICKLE

In [ ]:
# Criação do dataframe pandas
df = pd.DataFrame(listaUtentes.generateDataset())
#listaUtentes = None # Limpa listaUtentes depois de gerar Dataset

## Limpeza das variáveis

Substituição de `None` por 0's e correcção do Types

In [ ]:
# Substitui por 0 todos Nones nas colunas que 
# começam por NrAnulacoesPorMotivo ou NrApresentacoesComResultado
filter_col = [col for col in df if col.startswith('NrAnulacoesPorMotivo') 
                                   or col.startswith('NrApresentacoesComResultado')]

for col in filter_col:
    df[col].fillna(value=0, inplace = True);
    df[col]=df[col].astype('int');

In [ ]:
df['CursoAnoConclusao'].fillna(value=np.nan, inplace = True)
df['NrDescendentesCargo'].fillna(value=np.nan, inplace = True)
df['NrDescendentesCargo'].fillna(value=0, inplace = True)

In [ ]:
df['Empregado'] = df['Empregado'].astype('category')
df['Idade'] = df['Idade'].astype('int')
df['NrDescendentesCargo'] = df["NrDescendentesCargo"].str.replace(",", ".").fillna(0)
df['NrDescendentesCargo'] = df["NrDescendentesCargo"].fillna(0).astype('float').astype('int')

df['NrPessoasCargo'] = df["NrPessoasCargo"].fillna(0).astype('float').astype('int')

In [ ]:
pf.ProfileReport(df)

## Preparação dos dados

In [ ]:
dfM = df[df.Empregado=='S']

In [ ]:
colNamesToDummify = ['Sexo', 
                     'RSI', 
                     'CategoriaAtrib', 
                     'Empregado', 
                     'EstadoCivil', 
                     'FormacaoProf', 
                     'HabilitacaoEscolar',
                     'Nacionalidade',
                     'Qualificacao']
for colName in colNamesToDummify:
    df_dummies = pd.get_dummies(dfM[colName], prefix=colName+'')
    dfM = pd.concat([dfM, df_dummies], axis = 1)
    dfM.drop(colName, axis = 1)
    
colNamesToDrop = ['AreaCurso', 
                  'AreaFormacao', 
                  'CnpAnterior', 
                  'ConjugeMotivoIndisponibilidade',
                  'CursoEstabelecimentoEnsinoSup',
                  'Data',
                  'DataNascimento',
                  'LTU',
                  'Utente',
                  'CursoAnoConclusao']

dfM.drop(colNamesToDummify, axis = 1, inplace = True)
dfM.drop(colNamesToDrop, axis = 1, inplace = True)

In [90]:
dfM.dtypes

DiasDesemprego                                                                                                        int64
Idade                                                                                                                 int32
NivelDeficiencia                                                                                                      int64
NrAnulacoes                                                                                                           int64
NrAnulacoesPorMotivo_CADUCIDADE DO DOCUMENTO DE IDENTIFICAÇÃO                                                         int32
NrAnulacoesPorMotivo_COLOCAÇÃO - CANDIDATURA EXTERNA                                                                  int32
NrAnulacoesPorMotivo_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA DE OUTREM                                                int32
NrAnulacoesPorMotivo_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA PRÓPRIA                                                  int32
NrAnulac

## Modelo LASSO

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

#Split data in train set and test set

y = dfM['DiasDesemprego']
X = dfM.drop('DiasDesemprego', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# #############################################################################
# Lasso
from sklearn.linear_model import Lasso

alpha = 0.01
lasso = Lasso(alpha=alpha, max_iter=100000, normalize = False)

y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
r2_score_lasso = r2_score(y_test, y_pred_lasso)
print(lasso)
print("r^2 on test data : %f" % r2_score_lasso)

In [26]:
X_train.dtypes

Idade                                                                                                                 int32
NivelDeficiencia                                                                                                      int64
NrAnulacoes                                                                                                           int64
NrAnulacoesPorMotivo_CADUCIDADE DO DOCUMENTO DE IDENTIFICAÇÃO                                                         int32
NrAnulacoesPorMotivo_COLOCAÇÃO - CANDIDATURA EXTERNA                                                                  int32
NrAnulacoesPorMotivo_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA DE OUTREM                                                int32
NrAnulacoesPorMotivo_COLOCAÇÃO POR MEIOS PRÓPRIOS, POR CONTA PRÓPRIA                                                  int32
NrAnulacoesPorMotivo_CORRECÇÃO COLOCAÇÃO FME                                                                          int32
NrAnulac